In [42]:
from utils import get_test_table
import numpy as np

In [43]:
X_train, X_test, y_train, y_test = get_test_table()
X_train = X_train.drop(["Unnamed: 0"], axis=1)
X_test=X_test.drop(["Unnamed: 0"], axis=1)

Encoding the date column

In [44]:
import pandas as pd

# Assuming 'X_train' and 'X_test' are pandas DataFrames and have a date column named 'date'
X_train['date'] = pd.to_datetime(X_train['date'])
X_test['date'] = pd.to_datetime(X_test['date'])

# Sort data by date
X_train = X_train.sort_values(by='date')
X_test = X_test.sort_values(by='date')

# Assign a unique number to each date
unique_dates = pd.concat([X_train['date'], X_test['date']]).unique()
date_to_number = {date: i for i, date in enumerate(sorted(unique_dates))}
X_train['date'] = X_train['date'].map(date_to_number).astype('float32')
X_test['date'] = X_test['date'].map(date_to_number).astype('float32')

# Convert the rest of your dataframe to float32
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Reshape for LSTM input
X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

Model

In [45]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras import backend as K

def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=[rmse])

model.fit(X_train, y_train, epochs=10, batch_size=72, validation_data=(X_test, y_test), verbose=2, shuffle=False)

Epoch 1/50
149/149 - 3s - loss: 79.3668 - rmse: 8.7272 - val_loss: 71.0511 - val_rmse: 8.2453 - 3s/epoch - 18ms/step
Epoch 2/50
149/149 - 1s - loss: 70.7653 - rmse: 8.2252 - val_loss: 65.0762 - val_rmse: 7.8860 - 528ms/epoch - 4ms/step
Epoch 3/50
149/149 - 0s - loss: 66.0759 - rmse: 7.9435 - val_loss: 62.0521 - val_rmse: 7.7035 - 473ms/epoch - 3ms/step
Epoch 4/50
149/149 - 0s - loss: 63.8721 - rmse: 7.8126 - val_loss: 60.6814 - val_rmse: 7.6241 - 467ms/epoch - 3ms/step
Epoch 5/50
149/149 - 0s - loss: 62.9173 - rmse: 7.7588 - val_loss: 60.1540 - val_rmse: 7.5970 - 463ms/epoch - 3ms/step
Epoch 6/50
149/149 - 0s - loss: 62.5620 - rmse: 7.7413 - val_loss: 59.9875 - val_rmse: 7.5909 - 464ms/epoch - 3ms/step
Epoch 7/50
149/149 - 0s - loss: 62.4499 - rmse: 7.7375 - val_loss: 59.9479 - val_rmse: 7.5911 - 479ms/epoch - 3ms/step
Epoch 8/50
149/149 - 0s - loss: 62.4208 - rmse: 7.7375 - val_loss: 59.9435 - val_rmse: 7.5923 - 472ms/epoch - 3ms/step
Epoch 9/50
149/149 - 0s - loss: 62.4151 - rmse: 7.

In [48]:
# Evaluate the model using RMSE
train_loss, train_rmse = model.evaluate(X_train, y_train, verbose=0)
test_loss, test_rmse = model.evaluate(X_test, y_test, verbose=0)
print(f'Train Loss: {train_loss}, Train RMSE: {train_rmse}')
print(f'Test Loss: {test_loss}, Test RMSE: {test_rmse}')

Train Loss: 62.41313171386719, Train RMSE: 7.598634243011475
Test Loss: 59.95191955566406, Test RMSE: 7.438716411590576
